In [12]:
# kaggle dependencies
########################
# pip install fastkaggle
# pip install wandb
# pip install polars
# pip install datasets
# pip install scikit-learn
# pip install evaluate

In [ ]:
# | default_exp 00_base_run

In [14]:
#| export 
import os

iskaggle = os.path.exists("/kaggle/input")
isremote = os.path.exists("/home/ubuntu")

In [15]:
#| export 
import wandb
import os
import shutil
import fastkaggle
from pathlib import Path
import numpy as np
import pandas as pd
import polars as pl
from datasets import Dataset
import torch  # base
import torch.nn.functional as F
import json
from transformers.trainer import Trainer
from transformers.training_args import TrainingArguments
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
)
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset as TorchDataset
import evaluate
from torch.utils.data import DataLoader
from datetime import datetime
import subprocess
from sklearn.metrics import accuracy_score

## LLM Response Scoring with BERT

This notebook is for the [llm-classification-finetuning](https://www.kaggle.com/competitions/llm-classification-finetuning) competition on kaggle. It's a quick fine-tune of the `bert-base-uncased` model to predict which LLM response is preferrable. There are probably better models and approaches for this, but BERT does pretty well on its own without a whole lot of intervetion.

For me, this was more of a quick experiment in getting some external dependincies set up in a kaggle `code competition` notebook.

- To get additional libraries installed, open the notebook in kaggle and select `Install Dependencies` from the `Add-On` menu.

- To run the BERT model offline, add this dataset to your notebook dependencies:
  - https://www.kaggle.com/datasets/xhlulu/huggingface-bert

You can see later on how to reference the local model location in the kaggle notebook.


Handle the GPU handoff for all the different machines


In [16]:
#| export 

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS (Apple Silicon GPU)")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA (NVIDIA GPU)")
else:
    device = torch.device("cpu")
    print("Using CPU")

print(f"Device: {device}")

Using CUDA (NVIDIA GPU)
Device: cuda


In [17]:
#| export 
if not iskaggle and not isremote:
    data_base_path = Path("./data")
    comp_name = "llm-classification-finetuning"
    datapath = data_base_path / comp_name
    if not os.path.exists(datapath) and not datapath.exists():
        install_path = fastkaggle.setup_comp(comp_name)
        shutil.move(install_path, datapath)

In [18]:
#| export 
if isremote:
    if os.getcwd() == "/home/ubuntu":
        os.chdir("./llm-classification-finetuning")

### Set up Kaggle/Local Env


In [19]:
#| export 
WANDB_PROJECT_NAME = "kaggle-llm-classification"


def setup_environment():
    """Detect environment and set up paths for both local and Kaggle"""

    if iskaggle:
        print("Running on Kaggle")

        INPUT_DIR = "/kaggle/input/llm-classification-finetuning"
        OUTPUT_DIR = "/kaggle/working"
        MODEL_DIR = "/kaggle/working/models"

        os.environ["WANDB_MODE"] = "disabled"
    else:
        print("💻 Running locally")

        INPUT_DIR = "./data/llm-classification-finetuning"
        OUTPUT_DIR = "./output"
        MODEL_DIR = "./models"

        os.environ["WANDB_PROJECT"] = WANDB_PROJECT_NAME
        os.environ["WANDB_LOG_MODEL"] = "false"
        os.environ["WANDB_WATCH"] = "false"
        # os.environ["WANDB_MODE"] = "disabled"

    os.makedirs(OUTPUT_DIR, exist_ok=True)
    os.makedirs(MODEL_DIR, exist_ok=True)
    # to kill warning when running in notebooks
    os.environ["TOKENIZERS_PARALLELISM"] = "false"

    return INPUT_DIR, OUTPUT_DIR, MODEL_DIR


INPUT_DIR, OUTPUT_DIR, MODEL_DIR = setup_environment()
print(f"Input directory: {INPUT_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Model directory: {MODEL_DIR}")

💻 Running locally
Input directory: ./data/llm-classification-finetuning
Output directory: ./output
Model directory: ./models


### Load Data


In [20]:
#| export 
try:
    df_train = pl.read_csv(f"{INPUT_DIR}/train.csv")
    df_test = pl.read_csv(f"{INPUT_DIR}/test.csv")
    df_sample = pl.read_csv(f"{INPUT_DIR}/sample_submission.csv")

    print("Data loaded successfully!")
    print(f"Train: {df_train.shape}")
    print(f"Test: {df_test.shape}")
    print(f"Sample submission: {df_sample.shape}")

except FileNotFoundError as e:
    print(f" Data file not found: {e}")
    print(f" Make sure data is in: {INPUT_DIR}")

    if os.path.exists(INPUT_DIR):
        files = os.listdir(INPUT_DIR)
        print(f"📁 Files in {INPUT_DIR}: {files}")

Data loaded successfully!
Train: (57477, 9)
Test: (3, 4)
Sample submission: (3, 4)


In [21]:

pl.Config.set_tbl_width_chars(300)
pl.Config.set_fmt_str_lengths(1000)

polars.config.Config

In [22]:
df_train.sample(n=10)

id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
i64,str,str,str,str,str,i64,i64,i64
922666394,"""gpt-3.5-turbo-0613""","""gpt-3.5-turbo-1106""","""[""Write a brief note about sports performance tracking, its benefits and uses""]""","""[""Sports performance tracking refers to the collection and analysis of data related to an athlete's performance in various sports activities. It involves the use of technology, such as wearable devices and software applications, to monitor and measure key performance metrics.\n\nThe benefits of sports performance tracking are numerous. Firstly, it allows athletes and coaches to gain a deeper understanding of an athlete's strengths and weaknesses. By tracking metrics such as speed, distance covered, heart rate, and power output, athletes can identify areas for improvement and develop targeted training programs.\n\nAdditionally, performance tracking enables athletes to set realistic goals and track their progress over time. By monitoring performance metrics, athletes can see their improvements and make adjustments to their training regimen accordingly.\n\nSports performance tracking also provides valuable insights for coaches and trainers. They can use the data to assess an athlete's per…","""[""Sports performance tracking involves monitoring and analyzing various metrics related to an athlete's performance, such as speed, endurance, strength, and skill level. This data can be collected through wearable devices, video analysis, and other tools to provide valuable insights into an athlete's progress and areas for improvement. The benefits of sports performance tracking include the ability to identify strengths and weaknesses, track progress over time, optimize training programs, prevent injuries, and enhance overall performance. Coaches, athletes, and sports organizations can use this data to make informed decisions about training, strategy, and player development. Overall, sports performance tracking is a valuable tool for improving athletic performance and maximizing potential.""]""",1,0,0
2822613620,"""mistral-medium""","""gpt-4-0314""","""[""Design a futuristic arena.""]""","""[""The OmniSphere Arena: A Futuristic Masterpiece\n\nThe OmniSphere Arena is a state-of-the-art, futuristic sports and entertainment venue that pushes the boundaries of design, technology, and sustainability. Located in the heart of Neo-Eden city, this architectural marvel can host a wide range of events, from high-intensity sports matches to mesmerizing concerts and immersive virtual reality experiences.\n\nStructure and Design:\n\nThe OmniSphere Arena features a sleek, spherical design with an outer shell made of advanced, transparent smart glass. This material can change its opacity and color, allowing for dynamic light shows and projections that create an ever-changing visual spectacle. The arena's diameter spans an impressive 500 meters, providing ample space for seating, amenities, and various event configurations.\n\nSustainability and Energy:\n\nIn line with Neo-Eden's commitment to environmental sustainability, the OmniSphere Arena is designed to minimize its carbon footprint. …","""[""The Futuristic Arena - \""The Nexus Coliseum\""\n\nExterior:\nThe Nexus Coliseum is a stunning architectural marvel with a diameter of 2 kilometers, housed within a floating semi-transparent geodesic dome. This dome is made of a self-cleaning, solar panel-infused smart glass that harnesses sunlight to power the arena. The exterior is adorned with holographic projections and LED screens displaying upcoming events, advertisements, and real-time statistics.\n\nEntrance:\nThe main entrance features a spacious, oval-shaped atrium with a holographic welcome projection and interactive touch kiosks for ticketing and information. The walls are lined with vertical gardens and bioluminescent plants, providing a sustainable and visually appealing environment. Visitors are guided to their seats via levit

In [23]:
df_train["prompt"][0]

'["Is it morally right to try to have a certain percentage of females on managerial positions?","OK, does pineapple belong on a pizza? Relax and give me fun answer."]'

Try concatenating full conversations and full answer sets


In [24]:
#| export 
cols = ["prompt", "response_a", "response_b"]
if isinstance(df_train["prompt"][0], str):
    first_prompt_cell = df_train["prompt"][0]
    if "[" in first_prompt_cell and "]" in first_prompt_cell:
        df_train = df_train.with_columns([pl.col(col).str.json_decode() for col in cols])
        # explode
        df_train = df_train.explode(cols)
        # join
        # df_train = df_train.with_columns([pl.col(col).list.join(" ") for col in cols])

In [25]:
df_train["prompt"][0]

'Is it morally right to try to have a certain percentage of females on managerial positions?'

Remove any empty conversational turns:


In [26]:
#| export 
def null_clean(df: pl.DataFrame):
    df = df.drop_nulls(subset=["prompt", "response_a", "response_b"])
    return df

In [27]:
#| export 
df_train = null_clean(df_train)
df_test = null_clean(df_test)

In [28]:
df_train

id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
i64,str,str,str,str,str,i64,i64,i64
30192,"""gpt-4-1106-preview""","""gpt-4-0613""","""Is it morally right to try to have a certain percentage of females on managerial positions?""","""The question of whether it is morally right to aim for a certain percentage of females in managerial positions is a complex ethical issue that involves considerations of fairness, equality, diversity, and discrimination. Here are some arguments in favor of and against such policies: **Arguments in favor:** 1. **Correcting Historical Inequities:** Women have historically been underrepresented in leadership roles due to various cultural, institutional, and social barriers. Aiming for a specific percentage can be seen as a corrective measure to address past and ongoing discrimination. 2. **Promoting Diversity:** Diverse leadership teams can enhance decision-making and represent a broader range of perspectives. This can lead to better outcomes for organizations and society as a whole. 3. **Equality of Opportunity:** Setting targets for female representation in management can help ensure that women have equal opportunities to advance in their careers. 4. **Role Modeling:** Increased v…","""As an AI, I don't have personal beliefs or opinions. However, I can tell you that the question of gender quotas in managerial positions is a complex one and positions can vary. Advocates argue that such measures can help correct historical imbalances, promote diversity, and may lead to better decision-making. Critics may argue that positions should be based purely on merit, and that quotas could potentially lead to tokenism or unqualified candidates being promoted. Morality can be subjective and differs from person to person based on their personal beliefs, cultural background, and values. It's a topic with valid arguments on both sides, and it's up to each individual or organization to decide what they believe is right.""",1,0,0
30192,"""gpt-4-1106-preview""","""gpt-4-0613""","""OK, does pineapple belong on a pizza? Relax and give me fun answer.""","""Ah, the age-old culinary conundrum that has divided nations and dinner tables: does pineapple belong on a pizza? The tropical twist of pineapple on pizza, known as Hawaiian pizza, is a hotly debated topic where taste buds battle and pizza purists protest. Let's slice into the debate with a zest of fun: **Team Pineapple:** ""Absolutely, yes! Pineapple on pizza is like a beach party in your mouth. The sweet juiciness of pineapple chunks frolicking with savory ham or bacon creates a flavor wave that surfs across the cheesy ocean of deliciousness. It's the Mardi Gras of pizzas, where sweet meets savory in a jubilant jamboree!"" **Team No-Pineapple:** ""No way, not in a million pizzas! Pineapple is a fruit that should be sunbathing on a tropical fruit platter, not freeloading on a sacred slice of pizza. The thought of warm, melty cheese conspiring with pineapple's sugary mischief is enough to make Italian ancestors turn in their tomato patches. Keep the pineapple in the fruit salad where it…","""As an AI, I don't eat, so I don't have personal preferences. But, in the world of pizza, the pineapple topping is a hotly debated topic! Some people love the sweet and savory mix it brings to a pizza, a flavor combination made famous by the Hawaiian pizza. Others firmly believe that the sweetness of the fruit has no place on a pizza. It's a pizza puzzle that might never be solved. So, whether pineapple belongs on a pizza or not, it truly depends on your taste buds!""",1,0,0
53567,"""koala-13b""","""gpt-4-0613""","""What is the difference between marriage license and marriage certificate?""","""A marriage license is a legal document that allows a couple to get married. It is issued by a government agency, such as a county clerk's office or a state government, and is valid for a certain period of time, usually one year. After the marriage has ta

Format for `BERT` by concatenating the prompt and both responses with `[SEP]` tokens in between


In [29]:
("[CLS]" + df_train["prompt"] + "[SEP]" + df_train["response_a"] + "[SEP]" + df_train["response_b"])[0]

'[CLS]Is it morally right to try to have a certain percentage of females on managerial positions?[SEP]The question of whether it is morally right to aim for a certain percentage of females in managerial positions is a complex ethical issue that involves considerations of fairness, equality, diversity, and discrimination.\n\nHere are some arguments in favor of and against such policies:\n\n**Arguments in favor:**\n\n1. **Correcting Historical Inequities:** Women have historically been underrepresented in leadership roles due to various cultural, institutional, and social barriers. Aiming for a specific percentage can be seen as a corrective measure to address past and ongoing discrimination.\n\n2. **Promoting Diversity:** Diverse leadership teams can enhance decision-making and represent a broader range of perspectives. This can lead to better outcomes for organizations and society as a whole.\n\n3. **Equality of Opportunity:** Setting targets for female representation in management can

In [30]:
#| export 
def apply_bert_fmt(df: pl.DataFrame):
    df = df.with_columns(
        (
           "[CLS]" + pl.col("prompt") + "[SEP]" + pl.col("response_a") + "[SEP]" + pl.col("response_b")
        ).alias("text")
    )
    return df

Condense to a single target column and remap to 0,1,2 as possible dep vars


In [31]:
#| export 
df_train = df_train.with_columns(
    pl.when(pl.col("winner_model_a") == 1)
    .then(pl.lit(0))
    .when(pl.col("winner_model_b") == 1)
    .then(pl.lit(1))
    .when(pl.col("winner_tie") == 1)
    .then(pl.lit(2))
    .otherwise(pl.lit(None))
    .alias("label")
)

In [32]:
#| export 
df_train = apply_bert_fmt(df_train)
df_test = apply_bert_fmt(df_test)

Split the training data into training and validation sets


In [33]:
df_train.sample(n=10)

id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,label,text
i64,str,str,str,str,str,i64,i64,i64,i32,str
1205976536,"""gpt-4-0314""","""wizardlm-13b""","""should I use ROS or ROS2 ""","""The choice between ROS (Robot Operating System) and ROS2 depends on your specific requirements and project goals. ROS (ROS1): 1. Mature and stable, with a large user community and extensive package support. 2. Works well for applications where real-time control is not critical. 3. Primarily supports Ubuntu, but has experimental support for other platforms. 4. Uses a custom build system (catkin) and communication middleware (roscpp). ROS2: 1. A newer version of ROS, with improvements in architecture, flexibility, and performance. 2. Real-time control capabilities, making it suitable for safety-critical applications. 3. Supports multiple platforms, including Ubuntu, macOS, and Windows. 4. Uses standard build tools (CMake, ament) and communication middleware (DDS). If you are starting a new project and require real-time control, cross-platform support, and improved performance, ROS2 would be a better choice. However, if you are already familiar with ROS1 and are working on a non-critic…","""If you have already been working with ROS and are familiar with the ecosystem, it might be easier to continue using ROS1. However, if you are new to ROS and want to take advantage of the latest features and improvements, ROS2 might be a better choice. ROS2 is designed to be more modular and easier to use than ROS1, and it also includes new tools and libraries that can help you build more robust and efficient robotic systems. Ultimately, the choice between ROS1 and ROS2 will depend on your specific needs and preferences, as well as the resources and support available to you.""",0,1,0,1,"""[CLS]should I use ROS or ROS2 [SEP]The choice between ROS (Robot Operating System) and ROS2 depends on your specific requirements and project goals. ROS (ROS1): 1. Mature and stable, with a large user community and extensive package support. 2. Works well for applications where real-time control is not critical. 3. Primarily supports Ubuntu, but has experimental support for other platforms. 4. Uses a custom build system (catkin) and communication middleware (roscpp). ROS2: 1. A newer version of ROS, with improvements in architecture, flexibility, and performance. 2. Real-time control capabilities, making it suitable for safety-critical applications. 3. Supports multiple platforms, including Ubuntu, macOS, and Windows. 4. Uses standard build tools (CMake, ament) and communication middleware (DDS). If you are starting a new project and require real-time control, cross-platform support, and improved performance, ROS2 would be a better choice. However, if you are already familiar with R…"
3468660601,"""wizardlm-13b""","""gpt-4-0613""","""how to grief loss of husband""","""Grieving the loss of a husband is a deeply personal and individual process, and there is no right or wrong way to do it. However, here are some general tips that may be helpful: 1. Allow yourself to feel your emotions: It's important to give yourself permission to feel sad, angry, or any other emotions that come up as you grieve. Don't try to suppress or ignore your feelings, as this can make them worse. 2. Seek support: Grieving can be isolating, but it's important to have a support system. This can include friends, family, a support group, or a therapist. 3. Take care of yourself: Grieving can be physically and emotionally exhausting. Make sure to take care of your physical health by eating well, getting enough sleep, and exercising (if safe and appropriate). 4. Be patient with yourself: Grieving is a process, and it takes time. Be patient with yourself and allow yourself to grieve at your own pace. 5. Honor your husband's memory: Find ways to honor your husband's memory, such as cr…","""I'm really sorry for your loss. Grieving is a personal process and everyone experience

In [34]:
#| export 
if iskaggle:
    model_path = "../input/huggingface-bert/bert-base-uncased"
    print("Loading BERT from Kaggle model input...")

else:
    model_path = "bert-base-uncased"
    print("Loading BERT from Hugging Face...")

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=3)
model.to(device)
model.device

Loading BERT from Hugging Face...


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


device(type='cuda', index=0)

In [35]:
#| export 
class TokenizeDataset(TorchDataset):
    def __init__(self, texts, tokenizer):
      self.texts = texts
      self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
       tokens = self.tokenizer.tokenize(self.texts[idx])
       return len(tokens)

In [36]:
#| export 
texts = df_train['text'].to_list()
dataset = TokenizeDataset(texts, tokenizer)
dataloader = DataLoader(dataset, batch_size=1000, num_workers=4)
token_counts = []
for batch in dataloader:
    token_counts.extend(batch.tolist())

Token indices sequence length is longer than the specified maximum sequence length for this model (714 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (576 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (894 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (738 > 512). Running this sequence through the model will result in indexing errors


In [37]:
# | export
df_train = df_train.with_columns(pl.Series("token_count", token_counts))

In [38]:
# | export
# for now just filter anything over the context window
df_train.filter(pl.col("token_count") < 510)

id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,label,text,token_count
i64,str,str,str,str,str,i64,i64,i64,i32,str,i64
53567,"""koala-13b""","""gpt-4-0613""","""What is the difference between marriage license and marriage certificate?""","""A marriage license is a legal document that allows a couple to get married. It is issued by a government agency, such as a county clerk's office or a state government, and is valid for a certain period of time, usually one year. After the marriage has taken place, the couple must obtain a marriage certificate, which is a document that records the marriage and is used to prove that the marriage took place. The marriage certificate is usually issued by the same government agency that issued the marriage license, and it is typically used for legal purposes, such as to change a name on a driver's license or to prove that a couple is married when applying for government benefits.""","""A marriage license and a marriage certificate are two different legal documents that have separate purposes. 1. Marriage License: A marriage license is a legal document that gives a couple permission to get married. It's usually obtained from local government or court officials before the wedding ceremony takes place. The couple is required to meet certain criteria, such as being of a certain age or not being closely related. Once the license is issued, there's often a waiting period before the marriage ceremony can take place. The marriage license has to be signed by the couple, their witnesses, and the officiant conducting the marriage ceremony, then returned to the license issuer for recording. 2. Marriage Certificate: A marriage certificate, on the other hand, is a document that proves a marriage has legally taken place. It's issued after the marriage ceremony, once the signed marriage license has been returned and recorded. The marriage certificate includes details about the cou…",0,1,0,1,"""[CLS]What is the difference between marriage license and marriage certificate?[SEP]A marriage license is a legal document that allows a couple to get married. It is issued by a government agency, such as a county clerk's office or a state government, and is valid for a certain period of time, usually one year. After the marriage has taken place, the couple must obtain a marriage certificate, which is a document that records the marriage and is used to prove that the marriage took place. The marriage certificate is usually issued by the same government agency that issued the marriage license, and it is typically used for legal purposes, such as to change a name on a driver's license or to prove that a couple is married when applying for government benefits.[SEP]A marriage license and a marriage certificate are two different legal documents that have separate purposes. 1. Marriage License: A marriage license is a legal document that gives a couple permission to get married. It's usually…",390
53567,"""koala-13b""","""gpt-4-0613""","""What is the minimal time to get them? 1 day or 10 days?""","""The minimal time to get a marriage license and a marriage certificate in California can vary depending on the specific requirements and process in your county. Generally, you can obtain a marriage license and a marriage certificate in California within a few days of applying, but it is best to check with your county clerk's office for the specific requirements and process for your county. In some counties, you can obtain a marriage license and a marriage certificate on the same day, while in other counties, you may need to wait a few days. The process typically takes about 30 minutes to complete and you will need to bring identification, such as a driver's license or passport, and proof of your age, such as a birth certificate or a passport. It is important to note that some counties in California require a waiting period between the time you obtain the marriage license and the time of the mar

In [39]:
#| export 
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_train["text"], df_train["label"], test_size=0.1, random_state=42
)

In [40]:
train_texts.shape, val_texts.shape, train_labels.shape, val_labels.shape

((64205,), (7134,), (64205,), (7134,))

In [41]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [42]:
tokenizer

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [43]:
ex_enc = tokenizer(train_texts[0], truncation=True, padding=True)
ex_enc

{'input_ids': [101, 101, 2632, 14855, 23940, 2527, 2003, 2013, 2073, 1029, 102, 2632, 14855, 23940, 2527, 2003, 1037, 4069, 3795, 2865, 2897, 2761, 2013, 26528, 1010, 1996, 3007, 2103, 1997, 12577, 1012, 2009, 2001, 3390, 2006, 2281, 1015, 1010, 2727, 1010, 2011, 1996, 23434, 1997, 12577, 2012, 1996, 2051, 1010, 12840, 10654, 4215, 8026, 27925, 2632, 2084, 2072, 1012, 1996, 2171, 1000, 2632, 14855, 23940, 2527, 1000, 16315, 2000, 1000, 1996, 6000, 1000, 1999, 2394, 1010, 7727, 2000, 1996, 13771, 6000, 1012, 2632, 14855, 23940, 2527, 2003, 2124, 2005, 2049, 2981, 1998, 2411, 4187, 6325, 1997, 2739, 1998, 2783, 3821, 1010, 2119, 2306, 1996, 2690, 2264, 1998, 2105, 1996, 2088, 1012, 102, 2632, 14855, 23940, 2527, 2003, 1037, 2865, 2897, 2241, 1999, 26528, 1010, 12577, 1012, 2009, 2001, 3390, 1999, 2727, 1998, 2038, 4961, 2000, 2022, 1037, 2350, 3795, 2739, 3029, 1010, 2007, 3674, 2694, 6833, 1998, 3617, 7248, 4346, 2739, 6325, 1999, 5640, 1010, 2394, 1010, 1998, 2060, 4155, 1012, 2295, 20

### Load Dataset


In [44]:
#| export 
class LLMDataset(torch.utils.data.Dataset):
    def __init__(self, texts, tokenizer, labels=None, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    # encoding on the fly here due to issues with memory on kaggle
    # when pre-tokenizing
    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        )

        item = {key: val.squeeze() for key, val in encoding.items()}

        if self.labels is not None:
            item["labels"] = torch.tensor(self.labels[idx])

        return item

    def __len__(self):
        return len(self.texts)

In [45]:
#| export 
train_dataset = LLMDataset(list(train_texts), tokenizer, list(train_labels))
val_dataset = LLMDataset(list(val_texts), tokenizer, list(val_labels))
test_dataset = LLMDataset(list(df_test["text"]), tokenizer)

In [46]:
#| export 
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, preds)}

### Train


In [ ]:
#| export 
final_model_path = f"{MODEL_DIR}/final"

timestamp = datetime.now().strftime("%Y%m%d-%H%M")
run_name = f"bert-classification-{timestamp}"
test_run = False
if not iskaggle and test_run:
    # Quick run to test pipeline
    training_args = TrainingArguments(
        output_dir=f"{OUTPUT_DIR}/results",
        run_name=run_name,
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        max_steps=2000,
        weight_decay=0.01,
        eval_strategy="no",
        save_strategy="no",
        load_best_model_at_end=False,
        metric_for_best_model="accuracy",
        logging_steps=2,
        logging_first_step=True,
        report_to="wandb" if os.environ.get("WANDB_MODE") != "disabled" else [],
        dataloader_num_workers=0,  # Important for Kaggle compatibility
    )
elif isremote: 
     training_args = TrainingArguments(
        output_dir=f"{OUTPUT_DIR}/results",
        run_name=run_name,
        learning_rate=2e-5,
        per_device_train_batch_size=128,
        per_device_eval_batch_size=128,
        num_train_epochs=2,
        weight_decay=0.01,
        eval_strategy="epoch",
        save_strategy="no",
        metric_for_best_model="accuracy",
        logging_steps=5,
        logging_first_step=True,
        report_to="wandb" if os.environ.get("WANDB_MODE") != "disabled" else [],
        dataloader_num_workers=0,  # Important for Kaggle compatibility
    ) 
else:
    training_args = TrainingArguments(
        output_dir=f"{OUTPUT_DIR}/results",
        run_name=run_name,
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=2,
        weight_decay=0.01,
        eval_strategy="epoch",
        save_strategy="no",
        metric_for_best_model="accuracy",
        logging_steps=5,
        logging_first_step=True,
        report_to="wandb" if os.environ.get("WANDB_MODE") != "disabled" else [],
        dataloader_num_workers=0,  # Important for Kaggle compatibility
    )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)
trainer.train()

# trainer.save_model(final_model_path)
# tokenizer.save_pretrained(final_model_path)
# if os.environ.get("WANDB_MODE") != "disabled":
#     wandb.log({"final_eval": eval_results})
#     wandb.save(f"{final_model_path}/*")
# wandb.finish()

### Inference


In [ ]:
# | export 
def load_model(model_path):
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    return model, tokenizer


model, tokenizer = load_model(final_model_path)
model.to(device)

text = "This is a test sentence"
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
outputs = model(**inputs.to(device))
predictions = outputs.logits

In [ ]:
predictions

In [ ]:
# | export 
preds = F.softmax(predictions, dim=-1)

In [ ]:
preds

In [ ]:
# | export 
all_probabilities = []
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

with torch.no_grad():
    for batch in test_dataloader:
        inputs = {k: v.to(model.device) for k, v in batch.items() if k != "labels"}

        outputs = model(**inputs)
        probabilities = F.softmax(outputs.logits, dim=-1)
        all_probabilities.extend(probabilities.cpu().numpy())
final_probs = np.vstack(all_probabilities)

In [ ]:
final_probs

In [ ]:
final_probs[:, 0]

In [ ]:
# | export 
submission_df = df_test
submission_df = submission_df.with_columns(
    pl.lit(final_probs[:, 0]).alias("winner_model_a"),
    pl.lit(final_probs[:, 1]).alias("winner_model_b"),
    pl.lit(final_probs[:, 2]).alias("winner_tie"),
)
submission_df = submission_df[["id", "winner_model_a", "winner_model_b", "winner_tie"]]
submission_df

In [ ]:
# | export 
df_for_kaggle = submission_df.to_pandas()

In [ ]:
# | export 
df_for_kaggle.to_csv("submission.csv", index=False)
df_for_kaggle.head()

In [49]:
from nbdev.export import nb_export

nb_export("20250709_unsplit_ds.ipynb", "llm_classifier")

---


### Push Notebook to Kaggle


In [ ]:
def push_notebook_cli():
    username = "peterbull"
    comp = "llm-classification-finetuning"
    notebook_file = "20250709_unsplit_ds.ipynb"
    metadata = {
        "id": f"{username}/llm-classification-bert-finetuning",
        "title": "LLM Classification BERT Finetuning",
        "code_file": notebook_file,
        "language": "python",
        "kernel_type": "notebook",
        "is_private": True,
        "enable_gpu": True,
        "enable_internet": False,  # required for kaggle code competition
        "dataset_sources": [],
        "competition_sources": [f"competitions/{comp}"],
        "kernel_sources": [],
    }

    with open("kernel-metadata.json", "w") as f:
        json.dump(metadata, f, indent=2)

    if not os.path.exists(notebook_file):
        print(" Notebook file not found!")
        print(" Files in current directory:")
        for f in os.listdir("."):
            if f.endswith(".ipynb"):
                print(f"{f}")
        return

    print("Pushing to Kaggle...")
    try:
        result = subprocess.run(
            ["kaggle", "kernels", "push", "-p", "."], capture_output=True, text=True, timeout=300
        )

        if result.returncode == 0:
            print("✅ Notebook pushed successfully!")
            print(result.stdout)
            print(
                f"🔗 View at: https://www.kaggle.com/code/{username}/llm-classification-bert-finetuning"
            )
        else:
            print("Error pushing notebook:")
            print(result.stderr)

    except subprocess.TimeoutExpired:
        print("Upload timed out after 5 minutes")
    except FileNotFoundError:
        print("Kaggle CLI not found. Install with: pip install kaggle")
    except Exception as e:
        print(f"Unexpected error: {e}")


if not iskaggle:
    push_notebook_cli()